# Introduction

Insomnia is one of the sleeping disorder. Although we may think that insomnia can occur only on adults, in fact it also can occur on adolescents. This data science project is delving into the insomnia during the adolescent period by using a dataset which contains the records of more than 90 adolecents. Individually, all participants have been following and participating in 19 questionnaires related to the sleep pattern, sleep habit, psychological issues, etc.

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

df = pd.read_csv(r"../datasets/insomnia_data.csv")
